In [1]:
%run "RBM_Classification.ipynb"

class DBN_Classification(RBM_Classification):
    # n_k is the number of classes, n_d is the # of dots representing one class
    def __init__(self, n_x, layers, n_k=10, n_d=1, k=1, mean_field=True):
        if len(layers) == 1:
            top_rbm_x = n_x
            lower_dbn = None
        else:
            top_rbm_x = layers[-2][0]
            lower_dbn = DBN(n_x, layers[:-1], k=k, mean_field=mean_field)
        
        if layers[-1][1] is not None:
            top_rbm = RBM_Classification.load_model(layers[-1][1])
            if top_rbm.n_x != top_rbm_x or top_rbm.n_k != n_k or top_rbm.n_h != layers[-1][0]:
                raise ValueError("Incorrect input model for the top classification layer") 
        else:
            top_rbm = RBM_Classification(top_rbm_x, layers[-1][0], n_k, n_d, k=k)

        self.mean_field = mean_field
        self.n_layer = len(layers)
        self.top_rbm = top_rbm
        self.lower_dbn = lower_dbn
     
        if lower_dbn:
            if top_rbm.training_done and not lower_dbn.training_done:
                     raise ValueError("Top layer is trained while lower layer is not.")            
            self.training_done = top_rbm.training_done and lower_dbn.training_done
        else:
            self.training_done = top_rbm.training_done

        return
        
    # this is the API for app to use
    def train(self, X, Label, epochs=1, batch_size=10, learning=0.01, save_file=None):
        if self.training_done: return
        
        save_file += ".dbn_classify.layer" + str(self.n_layer)
        lower_dbn = self.lower_dbn
        if lower_dbn is not None:
            if not lower_dbn.training_done:
                self.lower_dbn.train_model(X, epochs, batch_size, learning, save_file)
                self.lower_dbn.training_done = True
        
            Hp, Hs = self.lower_dbn.forward(X)
            if not self.mean_field:
                X = []; sampling = 5
                for j in range(sampling):
                    Vs = np.random.binomial(1, Hp, size=Hp.shape)
                    X.append(Vs)
                X = np.asarray(X).reshape(-1, Hp.shape[1])
                Label = np.asarray(list(Label)*sampling)
            else:
                X = Hp
            
        # train the top RBM
        if self.top_rbm.training_done: return
        
        save_file += "-"+str(self.n_layer)
        X = self.top_rbm.convert_input(X, Label)
        self.top_rbm.train_model(X, epochs, batch_size, learning, save_file)
        self.training_done = True
        return
        
    def classify(self, x):
        x = x.reshape(1,-1)
        if self.lower_dbn is not None:
            x, hs = self.lower_dbn.forward(x)

        vp, pred = self.top_rbm.classify(x)
        
        vp = vp.reshape(1,-1)
        if self.lower_dbn is not None:
            vp, vs = self.lower_dbn.backward(vp)

        return vp[0], pred
    
    def generate(self, label, k_gibbs, init_w, w):
        
        vp = self.top_rbm.generate(label, k_gibbs, init_w, w)
        
        vp = vp.reshape(1,-1)
        if self.lower_dbn is not None:
            vp, vs = self.lower_dbn.backward(vp)

        return vp[0]
    
    def show_features(self, shapes, count=-1):

        #for i in range(self.n_layer-1):
        #    rbm = self.rbms[i]
        #    rbm.show_features(shapes[i], "MNIST learned features in DBN layer %d" % (i+1), count)
        #
        #self.top_rbm.show_features(shapes[-1], "MNIST learned features in DBN layer %d" % (self.n_layer), count)
        '''
        lower_rbm = None
        for i in range(self.n_layer):
            rbm = self.rbms[i]
            if i!=0:
                W = np.dot(rbm.W, lower_rbm.W)
                rbm = RBM(W.shape[1], W.shape[0], W=W)
               
            rbm.show_features(shapes[0], "MNIST learned features in DBN layer %d" % (i+1), count)
            lower_rbm = rbm
        '''
        return
            

In [2]:
# testing of the RBM Classification_Generation code above

%run "mnist.ipynb"
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import Grid
import math

class MNIST_DBN_CLASSIFY(MNIST_RBM_CLASSIFY):
    def __init__(self, n_x, layers, n_k=10, n_d=1, mean_field=True, folder="../convolution-network"):
        self.train_input = MnistInput("train", folder)
        self.test_input = MnistInput("test", folder)
        self.net = DBN_Classification(n_x, layers, n_k, n_d, mean_field=mean_field)
        return
    '''
    def train(self, train_size=-1, n_epoch=100, batch_size=10, learning=0.1):
        X = []
        Y = []
        n_sample = 0
        for x, y in self.train_input.read(train_size):
            X.append(x)
            Y.append(y)
            n_sample += 1

        X = (np.array(X).reshape(n_sample, -1) > 30) * 1
        Label = np.array(Y).reshape(n_sample, -1)
        
        self.net.train(X, Label, n_epoch, batch_size, learning, mean_field=True, save_file="mnist")
        return
    '''

In [ ]:
%run "DBN.ipynb"
%run "RBM_Classification.ipynb"

mnist_dbn = None
if __name__ == "__main__" and '__file__' not in globals():
    np.seterr(all='raise')
    plt.close('all')

    v = (28,28); h1 = (20,25); h2 = (20,25); h3 = (561,1)
    
    layers = [
#        (mult(h1), "2mnist_dbn.layer1.epochs100.784x500"), # (dimension, "model_file") of hidden layer 1
        (mult(h1), None),  # (dimension, "model_file") of 2nd layer
#        (mult(h2), "2mnist_dbn.layer2.epochs100.500x500"), # (dimension, "model_file") of hidden layer 2
        (mult(h2), None),  # (dimension, "model_file") of 2nd layer
#        (mult(h3), "mnist_rbm_classify.(784+10)x561.epochs100")  # (dimension, "model_file") of top layer
        (mult(h3), None)  # (dimension, "model_file") of top layer
    ]

    mnist_dbn = MNIST_DBN_CLASSIFY(mult(v), layers, mean_field=False, folder="../convolution-network")
    mnist_dbn.train(train_size=-1, n_epoch=100, batch_size=10)
    mnist_dbn.net.show_features([v], 56)
    mnist_dbn.classify()
    mnist_dbn.generate(k_gibbs=10, init_w=9, w=10)